In [44]:
import spacy

In [45]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import spacy
import time
from datetime import datetime
from spacy.symbols import ORTH, LEMMA, POS

In [46]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.tweets={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        searchTime=datetime.now()
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",count=10)
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt,searchTime,term)
            time.sleep(5)
                
    def addTweet(self,tweet,searchTime,term="",count=0):
        id = tweet['id_str']
        if id not in self.tweets.keys():
            self.tweets[id]={}
            self.tweets[id]['tweet']=tweet
            self.tweets[id]['count']=0
            self.tweets[id]['searchTime']=searchTime
            self.tweets[id]['searchTerm']=term
        self.tweets[id]['count'] = self.tweets[id]['count'] +1
        
    def getTweet(self,id):
        if id in self.tweets:
            return self.tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.tweets[id]['count']
    
    def countTweets(self):
        return len(self.tweets)
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        for t,entry in self.tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.tweets.keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.tweets)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.tweets=incontents
        
    def getSearchTerm(self,id):
        return self.tweets[id]['searchTerm']
    
    def getSearchTime(self,id):
        return self.tweets[id]['searchTime']
    
    def getText(self,id):
        tweet = self.getTweet(id)
        text=tweet['full_text']
        if 'retweeted_status'in tweet:
            original = tweet['retweeted_status']
            text=original['full_text']
        return text
                
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
   
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
 
    def getCodes(self,id):
        tweet=self.getTweet(id)
        return tweet['codes']
    
    # NEW -ROUTINE TO GET PROFILE
    def getCodeProfile(self):
        summary={}
        for id in self.tweets.keys():
            tweet=self.getTweet(id)
            if 'codes' in tweet:
                for code in tweet['codes']:
                    if code not in summary:
                            summary[code] =0
                    summary[code]=summary[code]+1
        sortedsummary = sorted(summary.items(),key=operator.itemgetter(0),reverse=True)
        return sortedsummary

In [47]:
def getTwitterNLP():
    nlp = spacy.load('en')
    
    for word in nlp.Defaults.stop_words:
        lex = nlp.vocab[word]
        lex.is_stop = True
    
    special_case = [{ORTH: u'e-cigarette', LEMMA: u'e-cigarette', POS: u'NOUN'}]
    nlp.tokenizer.add_special_case(u'e-cigarette', special_case)
    nlp.tokenizer.add_special_case(u'E-cigarette', special_case)
    vape_case = [{ORTH: u'vape',LEMMA:u'vape',POS: u'NOUN'}]
    
    vape_spellings =[u'vap',u'vape',u'vaping',u'vapor',u'Vap',u'Vape',u'Vapor',u'Vapour']
    for v in vape_spellings:
        nlp.tokenizer.add_special_case(v, vape_case)
    def hashtag_pipe(doc):
        merged_hashtag = True
        while merged_hashtag == True:
            merged_hashtag = False
            for token_index,token in enumerate(doc):
                if token.text == '#':
                    try:
                        nbor = token.nbor()
                        start_index = token.idx
                        end_index = start_index + len(token.nbor().text) + 1
                        if doc.merge(start_index, end_index) is not None:
                            merged_hashtag = True
                            break
                    except:
                        pass
        return doc
    nlp.add_pipe(hashtag_pipe,first=True)
    return nlp

In [48]:
smoking=Tweets()
smoking.readTweets("tweets-smoking.json")
smoking.countTweets()

100

In [49]:
vaping=Tweets()
vaping.readTweets("tweets-vaping.json")
vaping.countTweets()

100

ok. so this is a bit opaque.  is sci-kit any clearer?

Trying the scikit spacy combination here ---

https://nicschrading.com/project/Intro-to-NLP-with-spaCy/

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
import string
import re

In [51]:
train = ["I love space. Space is great.", "Planets are cool. I am glad they exist in space", 
        "lol @twitterdude that is gr8", "twitter &amp; reddit are fun.", 
        "Mars is a planet. It is red.", "@Microsoft: y u skip windows 9?", 
        "Rockets launch from Earth and go to other planets.", "twitter social media &gt; &lt;", 
        "@someguy @somegirl @twitter #hashtag", "Orbiting the sun is a little blue-green planet."]
labelsTrain = ["space", "space", "twitter", "twitter", "space", "twitter", "space", "twitter", "twitter", "space"]

test = ["i h8 riting comprehensibly #skoolsux", "planets and stars and rockets and stuff"]
labelsTest = ["twitter", "space"]

In [52]:
vectorizer = CountVectorizer()

In [53]:
vectorizer.fit(train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [54]:
print(vectorizer.vocabulary_)

{'red': 32, 'orbiting': 28, 'blue': 4, 'sun': 40, 'in': 17, 'glad': 10, 'someguy': 38, 'exist': 7, 'earth': 6, 'to': 44, 'is': 18, 'great': 13, 'the': 42, 'other': 29, 'amp': 1, 'reddit': 33, 'social': 36, 'that': 41, 'microsoft': 27, 'it': 19, 'hashtag': 16, 'planets': 31, 'cool': 5, 'from': 8, 'launch': 20, 'planet': 30, 'are': 3, 'media': 26, 'fun': 9, 'somegirl': 37, 'green': 14, 'gt': 15, 'am': 0, 'space': 39, 'lol': 22, 'go': 11, 'little': 21, 'they': 43, 'rockets': 34, 'gr8': 12, 'twitterdude': 46, 'twitter': 45, 'love': 23, 'skip': 35, 'windows': 47, 'lt': 24, 'mars': 25, 'and': 2}


In [55]:
vectorizer.get_feature_names()

['am',
 'amp',
 'and',
 'are',
 'blue',
 'cool',
 'earth',
 'exist',
 'from',
 'fun',
 'glad',
 'go',
 'gr8',
 'great',
 'green',
 'gt',
 'hashtag',
 'in',
 'is',
 'it',
 'launch',
 'little',
 'lol',
 'love',
 'lt',
 'mars',
 'media',
 'microsoft',
 'orbiting',
 'other',
 'planet',
 'planets',
 'red',
 'reddit',
 'rockets',
 'skip',
 'social',
 'somegirl',
 'someguy',
 'space',
 'sun',
 'that',
 'the',
 'they',
 'to',
 'twitter',
 'twitterdude',
 'windows']

In [56]:
vectorizer.vocabulary_.get("windows")

47

In [57]:
vector=vectorizer.transform(train)

In [58]:
print(vector.shape)
print(type(vector))
print(vector.toarray())

(10, 48)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 2 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 1 0]
 [0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0
  0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 1 1 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 

In [59]:
len(train)

10

https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(train)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

{'red': 32, 'orbiting': 28, 'blue': 4, 'sun': 40, 'in': 17, 'glad': 10, 'someguy': 38, 'exist': 7, 'earth': 6, 'to': 44, 'is': 18, 'great': 13, 'the': 42, 'other': 29, 'amp': 1, 'reddit': 33, 'social': 36, 'that': 41, 'microsoft': 27, 'it': 19, 'hashtag': 16, 'planets': 31, 'cool': 5, 'from': 8, 'launch': 20, 'planet': 30, 'are': 3, 'media': 26, 'fun': 9, 'somegirl': 37, 'green': 14, 'gt': 15, 'am': 0, 'space': 39, 'lol': 22, 'go': 11, 'little': 21, 'they': 43, 'rockets': 34, 'gr8': 12, 'twitterdude': 46, 'twitter': 45, 'love': 23, 'skip': 35, 'windows': 47, 'lt': 24, 'mars': 25, 'and': 2}
[2.70474809 2.70474809 2.70474809 2.29928298 2.70474809 2.70474809
 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809
 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809
 1.78845736 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809
 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809
 2.29928298 2.29928298 2.70474809 2.70474809 2.70474809 2.70474809
 

In [61]:
vector = vectorizer.transform(train)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(10, 48)
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.43323568 0.         0.         0.         0.
  0.28646791 0.         0.         0.         0.         0.43323568
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.73657982 0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.34989927 0.         0.         0.29744635 0.         0.34989927
  0.         0.34989927 0.         0.         0.34989927 0.
  0.         0.         0.         0.         0.         0.34989927
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.29744635 0.         0.         0.         0.
  0.         0.         0.         0.29744635 0.         0.
  0.         0.34989927 0.         0.         0.         0

ok. now -- how can I train on this? 
and how can I pass in my spacy tokens?

# plan

1. Go back to https://nicschrading.com/project/Intro-to-NLP-with-spaCy/

1.1 create NLP as per part 3 with hashtag tokenizer.

1.2. wrap that in a tokenizer that calls the spacy NLP parser and tokenizes with stops, etc.

1.3 use that tokenizer in a tf-df

1.4 follow that up with classifier. 

1.5 test


In [62]:
nlp = spacy.load('en')
def hashtag_pipe(doc):
    merged_hashtag = True
    while merged_hashtag == True:
        merged_hashtag = False
        for token_index,token in enumerate(doc):
            if token.text == '#':
                try:
                    nbor = token.nbor()
                    start_index = token.idx
                    end_index = start_index + len(token.nbor().text) + 1
                    if doc.merge(start_index, end_index) is not None:
                        merged_hashtag = True
                        break
                except:
                    pass
    return doc
nlp.add_pipe(hashtag_pipe,first=True)

In [63]:
def includeToken(tok):
    val =False
    if tok.is_stop == False:
        if tok.is_alpha == True: 
            if tok.text =='RT':
                val = False
            elif tok.pos_=='NOUN' or tok.pos_=='PROPN' or tok.pos_=='VERB':
                val = True
        elif tok.text[0]=='#' or tok.text[0]=='@':
            val = True
    if val== True:
        stripped =tok.lemma_.lower().strip()
        if len(stripped) ==0:
            val = False
        else:
            val = stripped
    return val

In [64]:
def filterTweetTokens(tokens):
    filtered=[]
    for t in tokens:
        inc = includeToken(t)
        if inc != False:
            filtered.append(inc)
    return filtered

In [65]:
def tokenizeText(sample):
    tokens = nlp(sample)
    filtered=[]
    for t in tokens:
        inc = includeToken(t)
        if inc != False:
            filtered.append(inc)
    return filtered 

In [66]:
nlp= getTwitterNLP()

def filterTweetTokens2(tokens):
    filtered=[]
    for t in tokens:
        inc = includeToken(t)
        if inc != False:
            filtered.append(inc)
    for ent in tokens.ents:
        stripped =ent.text.strip()
        if len(stripped) >0:
            filtered.append(ent.text.strip())
    return filtered

def tokenizeText2(text):
    tokens=nlp(text)
    return filterTweetTokens2(tokens)

1.3 - set up tokenizer in a vectorizer.

In [67]:
vectorizer= TfidfVectorizer(tokenizer=tokenizeText)
vectorizer2 = TfidfVectorizer(tokenizer=tokenizeText2)
clf = LinearSVC()
clf2 = LinearSVC()
pipe = Pipeline([('vectorizer', vectorizer), ('clf', clf)])
pipe2= Pipeline([('vectorizer', vectorizer2), ('clf', clf2)])

1.4 now I would do vectorizer. fit - need good input and labels..

In [68]:
smoking

In [69]:
vaping

## ok. to do this I think the following needs to happen
1. flatten the lists into pairs of text and category
2. shuffle and pick 0.8 of each contaat

In [70]:
def flattenTweets(tweets):
    flat=[]
    for i in tweets.getIds():
        text = tweets.getText(i)
        cat = tweets.getSearchTerm(i) 
        pair =(text,cat)
        flat.append(pair)
    return flat

In [71]:
def getTestTrainSplit(pairs,splitFactor=0.8):
    random.shuffle(pairs)
    split=int(len(pairs)*splitFactor)
    train=pairs[:split]
    test =pairs[split:]
    return train,test

In [72]:
def getTestTrain(smoking,vaping):
    fsmoke=flattenTweets(smoking)
    fvape=flattenTweets(vaping)
    strain,stest=getTestTrainSplit(fsmoke)
    vtrain,vtest=getTestTrainSplit(fvape)
    train=strain+vtrain
    test=stest+vtest
    random.shuffle(train)
    random.shuffle(test)
    return train,test

In [73]:
# ok. split established, but something is not right. need to ensure right split. 
# an then will need to split train and test into cateogires and texts

In [74]:
train,test=getTestTrain(smoking,vaping)

In [75]:
len(train)

160

In [76]:
len(test)

40

In [77]:
trainTweets,trainCats=zip(*train)

In [78]:
len(trainTweets)

160

In [79]:
trainTweets[0]

'@Dunham2Times Stg that’s some cigarette smoking shit because I’d never. I always wonder what people’s parents would think of they seen them cutting a fool 😂'

In [80]:
len(trainCats)

160

In [81]:
trainCats[:10]

('smoking',
 'vaping',
 'vaping',
 'smoking',
 'vaping',
 'vaping',
 'smoking',
 'smoking',
 'smoking',
 'smoking')

now try to run through the pipe.

In [82]:
pipe.fit(trainTweets,trainCats)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

 ok. did that do anything? Let's get the test

In [83]:
testTweets,testCats=zip(*test)

In [84]:
preds = pipe.predict(testTweets)

In [85]:
print("accuracy:", accuracy_score(testCats, preds))

accuracy: 1.0


ok. 97.5% accurate classification. can we look at the mismatches?

zip up input, category, prediction.  Look at first few

In [86]:
res=zip(testTweets,testCats,preds)
reslist=list(res)
reslist[:10]

[('Smoking a BIG DICK if yu think I need yu😏', 'smoking', 'smoking'),
 ('What are the V2 Vape Pens https://t.co/vrNJsq9N27 Remember to use our #coupon SOFLA10 during checkout #Vape #Vaping #VapeBlast #RT #eCigs',
  'vaping',
  'vaping'),
 ('$26.50! SMOK Stick One Plus Starter Kit 2000mah Battery Micro TFV4 Plus Tank https://t.co/j09lhgWtTT #vape #ecig #vaping #vapelife #vapefam #ejuice #ecigbattery .@Cuecig https://t.co/BkPTWYvT4p',
  'vaping',
  'vaping'),
 ('RT MtBakerVapor Get #MtBakerVapor at https://t.co/cZo4rT0bgU "NOW AVAILABLE - yamivapor - 8 different flavors, sure to satisfy your taste buds! #vaping #ejuice\n.\nGet yours now: https://t.co/QH72YlTdZE https://t.co/i6JGyGy649" #ejuice #eliquid #vape',
  'vaping',
  'vaping'),
 ('Made a sandwich 10 min ago and been looking for it ever since then\U0001f926🏾\u200d♂️ I gotta stop smoking😂 https://t.co/NCbNOyvZXe',
  'smoking',
  'smoking'),
 ('Welcome to Eternity...Smoking or non smoking ? https://t.co/W9nAU7GEaQ',
  'smoking',
  's

filter out those that don't match

In [87]:
res=zip(testTweets,testCats,preds)
for (sample,label,pred) in res:
    if label != pred:
        print(sample+", "+label+", "+pred)

ok. must put this into the text to challenge people to make the record better.  Then, challenge them to annotate and filter..

simlar for pipe2

In [88]:
pipe2.fit(trainTweets,trainCats)
preds = pipe2.predict(testTweets)

In [89]:
print("accuracy:", accuracy_score(testCats, preds))

accuracy: 1.0


In [90]:
res=zip(testTweets,testCats,preds)
reslist=list(res)
reslist[:10]

[('Smoking a BIG DICK if yu think I need yu😏', 'smoking', 'smoking'),
 ('What are the V2 Vape Pens https://t.co/vrNJsq9N27 Remember to use our #coupon SOFLA10 during checkout #Vape #Vaping #VapeBlast #RT #eCigs',
  'vaping',
  'vaping'),
 ('$26.50! SMOK Stick One Plus Starter Kit 2000mah Battery Micro TFV4 Plus Tank https://t.co/j09lhgWtTT #vape #ecig #vaping #vapelife #vapefam #ejuice #ecigbattery .@Cuecig https://t.co/BkPTWYvT4p',
  'vaping',
  'vaping'),
 ('RT MtBakerVapor Get #MtBakerVapor at https://t.co/cZo4rT0bgU "NOW AVAILABLE - yamivapor - 8 different flavors, sure to satisfy your taste buds! #vaping #ejuice\n.\nGet yours now: https://t.co/QH72YlTdZE https://t.co/i6JGyGy649" #ejuice #eliquid #vape',
  'vaping',
  'vaping'),
 ('Made a sandwich 10 min ago and been looking for it ever since then\U0001f926🏾\u200d♂️ I gotta stop smoking😂 https://t.co/NCbNOyvZXe',
  'smoking',
  'smoking'),
 ('Welcome to Eternity...Smoking or non smoking ? https://t.co/W9nAU7GEaQ',
  'smoking',
  's

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

In [92]:
simtexts=["The team made their way thorugh the season."]

In [93]:
simvectorizer=CountVectorizer()
simvectorizer.fit(simtexts)
simvec =simvectorizer.transform(simtexts)
print(simvec.toarray())

[[1 1 1 2 1 1 1]]


In [94]:
print(simvectorizer.vocabulary_)

{'season': 1, 'thorugh': 5, 'made': 0, 'the': 3, 'their': 4, 'way': 6, 'team': 2}


In [95]:
%matplotlib inline

import operator
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import spacy
import time
from datetime import datetime
from spacy.symbols import ORTH, LEMMA, POS

In [96]:
def getTwitterNLP():
    nlp = spacy.load('en')
    
    for word in nlp.Defaults.stop_words:
        lex = nlp.vocab[word]
        lex.is_stop = True
    
    special_case = [{ORTH: u'e-cigarette', LEMMA: u'e-cigarette', POS: u'NOUN'}]
    nlp.tokenizer.add_special_case(u'e-cigarette', special_case)
    nlp.tokenizer.add_special_case(u'E-cigarette', special_case)
    vape_case = [{ORTH: u'vape',LEMMA:u'vape',POS: u'NOUN'}]
    
    vape_spellings =[u'vap',u'vape',u'vaping',u'vapor',u'Vap',u'Vape',u'Vapor',u'Vapour']
    for v in vape_spellings:
        nlp.tokenizer.add_special_case(v, vape_case)
    def hashtag_pipe(doc):
        merged_hashtag = True
        while merged_hashtag == True:
            merged_hashtag = False
            for token_index,token in enumerate(doc):
                if token.text == '#':
                    try:
                        nbor = token.nbor()
                        start_index = token.idx
                        end_index = start_index + len(token.nbor().text) + 1
                        if doc.merge(start_index, end_index) is not None:
                            merged_hashtag = True
                            break
                    except:
                        pass
        return doc
    nlp.add_pipe(hashtag_pipe,first=True)
    return nlp

def includeToken(tok):
    val =False
    if tok.is_stop == False:
        if tok.is_alpha == True: 
            if tok.text =='RT':
                val = False
            elif tok.pos_=='NOUN' or tok.pos_=='PROPN' or tok.pos_=='VERB':
                val = True
        elif tok.text[0]=='#' or tok.text[0]=='@':
            val = True
    if val== True:
        stripped =tok.lemma_.lower().strip()
        if len(stripped) ==0:
            val = False
        else:
            val = stripped
    return val

def filterTweetTokens(tokens):
    filtered=[]
    for t in tokens:
        inc = includeToken(t)
        if inc != False:
            filtered.append(inc)
    return filtered

In [97]:
def tokenizeText(text):
    nlp=getTwitterNLP()
    tokens=nlp(text)
    return filterTweetTokens(tokens)

simvectorizer2=CountVectorizer(tokenizer=tokenizeText)

In [98]:
simvectorizer2.fit(simtexts)
simvec2 =simvectorizer2.transform(simtexts)
print(simvectorizer2.vocabulary_)
print(simvec2.toarray())

{'season': 0, 'way': 2, 'team': 1}
[[1 1 1]]


In [104]:
simvectorizer2.stop_words

In [105]:
simvectorizer3=CountVectorizer(tokenizer=tokenizeText,analyzer='word')
simvectorizer3.fit(simtexts)
simvec3=simvectorizer3.transform(simtexts)
print(simvectorizer3.vocabulary_)
print(simvec3.toarray())

{'season': 0, 'way': 2, 'team': 1}
[[1 1 1]]


In [109]:
simtexts

['The team made their way thorugh the season.']

In [110]:
tokenizeText(simtexts[0])

['team', 'way', 'season']

hmm. now it is DOING IT  right. 

In [113]:
texts2=["Welcome to Eternity...Smoking or non smoking ? https://t.co/W9nAU7GEaQ"]

In [116]:
simvectorizer=CountVectorizer()
simvectorizer.fit(texts2)
simvec =simvectorizer.transform(texts2)
print(simvectorizer.vocabulary_)
print(simvec.toarray())

{'to': 6, 'non': 3, 'smoking': 5, 'https': 2, 'eternity': 1, 'w9nau7geaq': 7, 'or': 4, 'welcome': 8, 'co': 0}
[[1 1 1 1 1 2 1 1 1]]


In [122]:
def preProcess(x):
    return x
simvectorizer3=CountVectorizer(tokenizer=tokenizeText,preprocessor=preProcess,analyzer='word')
simvectorizer3.fit(texts2)
simvec3=simvectorizer3.transform(texts2)
print(simvectorizer3.vocabulary_)
print(simvec3.toarray())

{'welcome': 2, 'smoking': 1, 'eternity': 0}
[[1 2 1]]


In [123]:
toks=tokenizeText(texts2[0])

In [124]:
toks

['welcome', 'eternity', 'smoking', 'smoking']

# ah. ah. somehow it's processing stuff tht pulls out 'welcome' why.

need to turn off the preprocessor...